In [1]:
! pip install pandas numpy scikit-learn statsmodels arch 
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.decomposition import FastICA
from statsmodels.tsa.stattools import adfuller
import arch
import warnings

warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_excel("dataset.xlsx")

In [3]:
dataset.rename(columns={'close_BBG0000776S2': 'close_B_1', 'close_BBG00B9PJ7V0': 'close_B_2', 'close_BBG00D6Q7LY6': 'close_B_3',\
                   'close_BBG00K53FBX6': 'close_B_4', 'close_BBG00R0Z4YW8': 'close_B_5',\
                   'coupon_BBG0000776S2': 'coupon_1', 'coupon_BBG00B9PJ7V0': 'coupon_2', 'coupon_BBG00D6Q7LY6': 'coupon_3',\
                   'coupon_BBG00K53FBX6': 'coupon_4', 'coupon_BBG00R0Z4YW8': 'coupon_5' }, inplace=True)

In [4]:
critical_columns = ['close_B_1', 'close_B_2', 'close_B_3', 'close_B_4', 'close_B_5',
       'close_AFKS', 'close_AGRO', 'close_BRENT', 'close_EUR_RUB__TOD',
       'close_GAZP', 'close_IMOEX', 'close_LKOH', 'close_NLMK', 'close_NVTK',
       'close_PLZL', 'close_ROSN', 'close_RTKM', 'close_RTSI', 'close_SBER',
       '%_0,25y', '%_0,5y', '%_0,75y', '%_1y', '%_2y',
       '%_3y', '%_5y', '%_7y', '%_10y', '%_15y', '%_20y', '%_30y']

В качестве риск-факторов м ывзяли цены закрытия ценных бумаг и валютных курсов доллара и  евро, что очевидно является риск-факторами нашего портфеля. Также мы взяли индексы российской фондовой биржи, как агрегирующие риск-факторы общего инвестиционного фона ФР, котировки нефти, от которых зависят как некоторые тикеры портфеля, так и  макроэкономика России, и ставки КБД по основным тенорам - риск-факторы процентных  ставок. Также мы учли и купоны по бондам, которые нужно будет продисконтировать что мы и сделаем ниже

In [5]:
print(len(dataset))
dataset = dataset.dropna(subset = critical_columns)
print(len(dataset))

1131
949


У нас есть столбцы купонов - которые конечно же влияют на стоимость облигаций. Давайте на каждую дату для каждой облигации посчитаем дисконтированную сумму купонов. По-хорошему, надо было для длинных бондов достать купоны вплоть до погашения - но мы немножко это опустим на милость проверяющего, дело ведь в технике исполнения а не мелочах)) Каждый купон будет продисконтирован по своему тенору КБД в зависимости от близости выплаты к рассматриваемой дате.

In [6]:
coupon_columns = ['coupon_1', 'coupon_2', 'coupon_3', 'coupon_4', 'coupon_5']
for col in coupon_columns:
    df[col] = df[col].replace([None], 0)
dataset.index = [i for i in range(len(dataset))]

In [21]:
def calculate_discounted_coupons(df, coup_col):
    res_col = []
    for index, row in df.iterrows():
        if index !=  len(df) -1 :
            diff_days = df['dt'].apply(lambda x: (x - row['dt']).days)
            mask = diff_days > 0 
            mask_1 = (0 < diff_days) & (diff_days <= 90)
            mask_2 = (90 < diff_days) & (diff_days <= 180)
            mask_3 = (180 < diff_days) & (diff_days <= 365)
            mask_4 = (365 < diff_days) & (diff_days <= 365*2)
            mask_5 = (365*2 < diff_days) & (diff_days <= 365*3)
            mask_6 = (365*3 < diff_days) & (diff_days <= 365*4)
            print(len(diff_days), len(mask), len(mask_1))
            total_discount = (df[coup_col]* mask*((mask_1 / (1 + row['%_0,25y'] / 365/100 )**(diff_days/365)) + \
                                                        (mask_2 / (1 + row['%_0,5y'] / 365/100 )**(diff_days/365)) +\
                                                        (mask_3 / (1 + row['%_1y'] / 365/100 )**(diff_days/365)) +\
                                                        (mask_4 / (1 + row['%_2y'] / 365/100 )**(diff_days/365)) +\
                                                        (mask_5 / (1 + row['%_3y'] / 365/100 )**(diff_days/365)) +\
                                                        (mask_6 / (1 + row['%_5y'] / 365/100 )**(diff_days/365))\
                                                        )).sum()
            print(total_discount)
        else: 
            total_discount = np.nan
        res_col.append(total_discount)
    df[f'discounted_{coup_col}'] = res_col
    # находим среднее значение столбца col без учета пропущенных значений и нулей
    col = f'discounted_{coup_col}'
    mean_value = df.loc[~df[col].isin([None, 0]), col].mean()

    # заполняем пропущенные значения и нули средним
    df[col] = df[col].replace([None, 0], mean_value)
    return df

In [22]:
for coup_col in coupon_columns:
    dataset = calculate_discounted_coupons(dataset, coup_col)

949 949 949
276.518080312211
949 949 949
276.51949467033575
949 949 949
276.52054151477523
949 949 949
276.5201126404064
949 949 949
276.51980642299424
949 949 949
276.52008108778534
949 949 949
276.52060495711044
949 949 949
276.52103988729505
949 949 949
276.52101945366746
949 949 949
276.5210602937114
949 949 949
276.521494289698
949 949 949
276.52180992954055
949 949 949
276.5215178923565
949 949 949
276.52179287100466
949 949 949
276.52212364233515
949 949 949
276.5221443583416
949 949 949
276.52199607596566
949 949 949
276.52216467949415
949 949 949
276.52270706791353
949 949 949
276.52309718277786
949 949 949
276.52310181009176
949 949 949
276.5237864591213
949 949 949
276.52454697016856
949 949 949
276.52488733530106
949 949 949
242.1153576601694
949 949 949
242.11563310855945
949 949 949
242.11632132558128
949 949 949
242.11706257637945
949 949 949
242.11711474464443
949 949 949
242.11774636764102
949 949 949
242.11790508683498
949 949 949
242.11792431128725
949 949 949
242.11

In [23]:
# больше нам не нужны столбцы купонов
cols_to_drop = ['coupon_1', 'coupon_2', 'coupon_3', 'coupon_4', 'coupon_5']
dataset = dataset.drop(columns=cols_to_drop)

In [24]:
print(len(dataset))
dataset = dataset.dropna()
print(len(dataset))

949
916


In [25]:
dataset.tail(2)

,dt,close_B_1,close_B_2,close_B_3,close_B_4,close_B_5,close_AFKS,close_AGRO,close_BRENT,close_EUR_RUB__TOD,...,%_7y,%_10y,%_15y,%_20y,%_30y,discounted_coupon_1,discounted_coupon_2,discounted_coupon_3,discounted_coupon_4,discounted_coupon_5
946,2023-12-27,705.98,844.40,917.48,818.22,837.4,16.054,1384.0,79.66,101.345,...,12.02,12.03,12.05,12.05,12.05,142.329043,179.452442,140.285598,151.176149,127.164799
947,2023-12-28,707.92,844.11,919.28,818.90,842.5,15.897,1367.0,77.53,99.800,...,11.84,11.80,11.74,11.68,11.62,142.329043,179.452442,140.285598,151.176149,127.164799


In [26]:
financial_columns = ['close_B_1', 'close_B_2', 'close_B_3', 'close_B_4', 'close_B_5',\
       'close_AFKS', 'close_AGRO', 'close_BRENT', 'close_EUR_RUB__TOD',\
       'close_GAZP', 'close_IMOEX', 'close_LKOH', 'close_NLMK', 'close_NVTK',\
       'close_PLZL', 'close_ROSN', 'close_RTKM', 'close_RTSI', 'close_SBER',\
       'close_USD000000TOD', '%_0,25y', '%_0,5y', '%_0,75y', '%_1y', '%_2y',\
       '%_3y', '%_5y', '%_7y', '%_10y', '%_15y', '%_20y', '%_30y'
       # ,\
       # 'discounted_coupon_1', 'discounted_coupon_2', 'discounted_coupon_3',\
       # 'discounted_coupon_4', 'discounted_coupon_5'
       ]
len(financial_columns)

32

Теперь сохраним датасет с полученными фичами

In [28]:
dataset.to_excel('dataset_coupons.xlsx', index = False)

Одной из альтернативных методов для снижения размерности данных, помимо метода главных компонент (PCA), является метод независимых компонент (Independent Component Analysis, ICA). Ниже приведены некоторые особенности метода ICA:

1. **Независимость компонент**: В отличие от PCA, который находит ортогональные компоненты (главные компоненты) с максимальной дисперсией, ICA стремится найти компоненты, которые являются независимыми друг от друга. Это позволяет выявить скрытые структуры данных, которые могут быть связаны с различными источниками.

2. **Способность к разделению источников**


3. **Робастность к негауссовским данным**

4. **Интерпретируемость компонент**

ICA может быть использован для разделения смешанных временных рядов, таких как цены акций или валютных курсов, на независимые составляющие. Это может помочь выявить скрытые структуры и зависимости между различными финансовыми инструментами.

То, что нужно!

In [ ]:
def get_risk_factors(dt_end = datetime.date(2023 , 12, 3), dataset = dataset, financial_columns = financial_columns):
    dt_end_datetime = pd.to_datetime(dt_end)
    # Фильтрация DataFrame по условию dt < dt_end
    dataset = dataset[dataset['dt'] < dt_end_datetime].reset_index().drop(columns = ['index'])
    df_financial = dataset[financial_columns]

    # Инициализация модели ICA
    ica = FastICA(n_components=10, algorithm='parallel', whiten=True, fun='logcosh', max_iter=300)

    # подгонка модели к данным и извлечение независимых компонент
    ica_components = ica.fit_transform(df_financial)

    # создание нового DataFrame с независимыми компонентами как столбцами
    df_ica = pd.DataFrame(data=ica_components)

    # вывод результата
    df_ica.columns = [f'risk_factor_{i}' for i in range(len(df_ica.columns))]
    df_ica['dt'] = dataset['dt']
    df_ica = df_ica[['dt'] +[f'risk_factor_{i}' for i in range(len(df_ica.columns)-1)]]
    # Объясненная дисперсия для каждой компоненты
    dataset = dataset.merge(df_ica, how = 'left', on = 'dt')

    return dataset


In [ ]:
df_risks = get_risk_factors(dt_end = datetime.date(2023 , 12, 3), dataset = dataset)

In [ ]:
df_risks.tail(2)

In [ ]:
df = df_risks[[f'risk_factor_{i}' for i in range(10)]].copy()
df.index = df_risks['dt']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
df.plot(figsize=(12, 6))  # Предполагается, что столбцы содержат временные данные и риск-факторы
plt.title('История значений риск-факторов')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.show()

# Описательная статистика
description = df.describe()
print(description)

# Корреляции риск-факторов
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Матрица корреляций риск-факторов')
plt.show()

# "Тяжесть хвостов" распределения
for column in df.columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Распределение {column}')
    plt.show()

    skew = df[column].skew()
    kurtosis = df[column].kurtosis()
    print(f'Риск-фактор: {column}; skewness: {skew}; kurtosis: {kurtosis}')

In [ ]:
for column in df.columns:
    decomposition = seasonal_decompose(df[column], model='additive', period=24)  # Предполагается, что есть сезонность

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    plt.figure(figsize=(12, 8))
    plt.subplot(411)
    plt.plot(df[column], label=f'Исходные данные {column}')
    plt.legend()

    plt.subplot(412)
    plt.plot(trend, label=f'Тренд {column}')
    plt.legend()

    plt.subplot(413)
    plt.plot(seasonal, label=f'Сезонность {column}')
    plt.legend()

    plt.subplot(414)
    plt.plot(residual, label=f'Остатки {column}')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
for column in df.columns:
    result = adfuller(df[column])
    print(f'Тест на стационарность для столбца {column}:')
    print(f'ADF статистика: {result[0]}')
    print(f'p-значение: {result[1]}')
    print('Критические значения:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')

    if result[1] <= 0.05:
        print(f'Результат: Ряд стационарен на уровне значимости 0.05')
    else:
        print('Результат: Ряд нестационарен')
    print()

По многим риск-факторам не наблюдается стационарность. Тогда применим для  них GARCH-модели. Используем библиотеку arch, его дефолтный метод fitа - ММП. Так как мы использовали ICA - можно использовать univariate models - ведь риск-факторы получены независимыми друг от друга.

In [ ]:
def garch_simulation(df, risk_factor_col, steps_forward, num_samples = 1000):
    model = arch.arch_model(df[risk_factor_col], mean = 'AR', vol='Garch', p=10, o=0, q=10)
    res = model.fit(disp='off')
    sim_data = res.forecast(horizon=steps_forward, method='simulation')
    means = sim_data.mean.iloc[0].values
    variances = sim_data.variance.iloc[0].values
    # Симуляция нормального распределения
    simulated_data = np.zeros((num_samples, len(means)))
    for i in range(len(means)):
        std_dev = np.sqrt(variances[i])
        simulated_data[:, i] = np.random.normal(means[i], std_dev, num_samples)
    return simulated_data[:steps_forward, :]


А теперь объединим расчёт риск-факторов и их симуляций для корректных бэк-тестов

In [ ]:
def calc_and_simulate_risk_factors(dt_end = datetime.date(2023 , 12, 3), steps_forward = 1, num_samples = 1000, dataset = dataset, financial_columns = financial_columns):
    df_risks = get_risk_factors(dt_end = dt_end, dataset = dataset, financial_columns = financial_columns)
    df = df_risks[[f'risk_factor_{i}' for i in range(10)]].copy()
    df.index = df_risks['dt']
    dict_of_risk_simulations = dict()
    for risk_factor_col in df.columns:
        dict_of_risk_simulations[risk_factor_col] = garch_simulation(df, risk_factor_col, steps_forward = steps_forward, num_samples = num_samples)
    return dict_of_risk_simulations

In [ ]:
dict_of_risk_simulations = calc_and_simulate_risk_factors(dt_end = datetime.date(2023 , 10, 3), num_samples = 1)